In [ ]:
pip install category_encoders

In [ ]:
import sqlite3
import pandas as pd
import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from category_encoders import OneHotEncoder 
import pickle
import joblib

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
file_path = '/content/BikeData.db'
conn = sqlite3.connect(file_path)
cur = conn.cursor()

In [ ]:
data = pd.read_sql("SELECT * FROM Bike WHERE Workingday='Yes'", conn)
data

,Id,Date,Hour,Temperature,Humidity,WIndspeed,Visibility,Dewpoint,Solarrad,Rain,Snow,Season,Holiday,Workingday,Rental_Count
0,1,2017-12-01,0,-5.2,37.0,2.2,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes,254
1,2,2017-12-01,1,-5.5,38.0,0.8,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes,204
2,3,2017-12-01,2,-6.0,39.0,1.0,2000,-17.7,0.0,0.0,0.0,Winter,No Holiday,Yes,173
3,4,2017-12-01,3,-6.2,40.0,0.9,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes,107
4,5,2017-12-01,4,-6.0,36.0,2.3,2000,-18.6,0.0,0.0,0.0,Winter,No Holiday,Yes,78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8460,8756,2018-11-30,19,4.2,34.0,2.6,1894,-10.3,0.0,0.0,0.0,Autumn,No Holiday,Yes,1003
8461,8757,2018-11-30,20,3.4,37.0,2.3,2000,-9.9,0.0,0.0,0.0,Autumn,No Holiday,Yes,764
8462,8758,2018-11-30,21,2.6,39.0,0.3,1968,-9.9,0.0,0.0,0.0,Autumn,No Holiday,Yes,694
8463,8759,2018-11-30,22,2.1,41.0,1.0,1859,-9.8,0.0,0.0,0.0,Autumn,No Holiday,Yes,712


In [ ]:
def engineer(df):
  # 비, 눈, 태양빛 수치를 하나의 기상으로 표현함
  df['weather'] = df['Solarrad'] - df['Rain'] - df['Snow']
  # Date 날짜 변환
  df['Date'] = pd.to_datetime(data['Date'])
  # 시계열 정렬
  df.sort_values(by=[data.columns[1], data.columns[2]], ascending=True, inplace=True)
  # 인덱스 초기화
  df.reset_index(drop=True, inplace=True)
  df['Month'] = df['Date'].dt.month
  df['Day'] = df['Date'].dt.day
  # 필요없는 column 제거
  df.drop(['Id', 'Solarrad', 'Rain', 'Snow', 'Season', 'Date', 'Workingday'], axis=1, inplace=True)
          
engineer(data)

In [ ]:
# 특성과 타겟 분리
target = data['Rental_Count']
feature = data.drop('Rental_Count', axis=1)
# One-Hot encoding
encoder = OneHotEncoder(use_cat_names=True)
feature_encoder = encoder.fit_transform(feature)

In [ ]:
# 시계열 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(feature_encoder, target, test_size=0.2, shuffle=False)

In [ ]:
# LinearRegression 회귀모델
model = LinearRegression()

# model fit
model.fit(X_train, y_train)

# 모델에 예측값 적용
y_pred = model.predict(X_test)

print('mean_squared_error : ', mean_squared_error(y_test, y_pred))

mean_squared_error :  239941.08311939976


In [ ]:
feature_encoder

,Hour,Temperature,Humidity,WIndspeed,Visibility,Dewpoint,Holiday_No Holiday,Holiday_Holiday,weather,Month,Day
0,0,-5.2,37.0,2.2,2000,-17.6,1,0,0.0,12,1
1,1,-5.5,38.0,0.8,2000,-17.6,1,0,0.0,12,1
2,2,-6.0,39.0,1.0,2000,-17.7,1,0,0.0,12,1
3,3,-6.2,40.0,0.9,2000,-17.6,1,0,0.0,12,1
4,4,-6.0,36.0,2.3,2000,-18.6,1,0,0.0,12,1
...,...,...,...,...,...,...,...,...,...,...,...
8460,19,4.2,34.0,2.6,1894,-10.3,1,0,0.0,11,30
8461,20,3.4,37.0,2.3,2000,-9.9,1,0,0.0,11,30
8462,21,2.6,39.0,0.3,1968,-9.9,1,0,0.0,11,30
8463,22,2.1,41.0,1.0,1859,-9.8,1,0,0.0,11,30


In [ ]:
# 임의의 데이터의 예측값 확인
model.predict([[2, 10, 40, 1.5, 1600, -12, 1, 0, 0.0, 3, 12]])

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  "X does not have valid feature names, but"


array([263.18417866])

In [ ]:
# 모델 피클링 / 모델 부호화
with open('bike_predict.pkl', 'wb') as pickle_file:
  pickle.dump(model, pickle_file)

In [ ]:
model1 = pickle.load(open('bike_predict.pkl','rb'))